# Trendline for model 

In [ ]:
from core.gsheet_utils import *
from core.sql_utils import get_sqlalchemy_engine
import pandas as pd
from sqlalchemy import text
from results.trendline.main import generate_trendline_functions
from core.sql_utils import *
from results.trendline.trendline_utils import filter_data, filter_trendlines
from results.trendline.main import load_all_data


### Load data

In [ ]:

engine = get_sqlalchemy_engine()

with engine.connect() as con:
    trendline_df = pd.read_sql(text("""SELECT vm.model_name, vm.id, vm.type, vm.version, m.make_name, b.capacity, vm.trendline, vm.trendline_max, vm.trendline_min FROM vehicle_model vm
                                  join make m on m.id=vm.make_id
                                  join battery b on b.id=vm.battery_id;"""), con)


In [ ]:
df = load_all_data()
df['origin'] = np.where(df['vin'].str.len() != 17, 'scrapping', 'production')


## Check conditioning

In [ ]:
model_ok = {}
for modele, group in df.groupby(["model_name", "type", 'vehicle_model_id']):
    print(modele)
    if filter_data(group, 'odometer', 'soh', km_lower=50_000, km_upper=50_000, vin_total=20, nbr_under=0, nbr_upper=0):
        model_ok[(modele[0], modele[1])] =  generate_trendline_functions(df[df['vehicle_model_id']==modele[2]], 'odometer', "soh")


In [ ]:
trendline_prod = trendline_df.dropna(subset=['trendline', 'trendline_max', 'trendline_min'])

In [ ]:
import plotly.graph_objects as go
from core.numpy_utils import numpy_safe_eval
def show_trendline_model(trendline, trendline_max, trendline_min, model, df=None):
    def trendline_apply(x, f):
        return numpy_safe_eval(f)
    fig = go.Figure()
    
    x_sorted = pd.Series([i*1000 for i in range(200)])
    fig.add_trace(
        go.Scatter(
            x=x_sorted,
            y=trendline_apply(x_sorted, trendline["trendline"]),
            mode="lines",
            line=dict(color="red", width=2),
            name="Trendline",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=x_sorted,
            y=trendline_apply(x_sorted, trendline_max["trendline"]),
            mode="lines",
            line=dict(color="green"),
            name="Upper",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=x_sorted,
            y=trendline_apply(x_sorted, trendline_min["trendline"]),
            mode="lines",
            line=dict(color="green"),
            name="Lower",
        )
    )
    
    # Ajouter les points de scrapping (en bleu)
    if df is not None and len(df) > 0:
        fig.add_trace(
            go.Scatter(
                x=df[df['origin']=='scrapping']['odometer'],
                y=df[df['origin']=='scrapping']['soh'],
                mode="markers",
                marker=dict(color="blue", size=8, opacity=0.7),
                name="Scrapping",
            )
        )
        fig.add_trace(
            go.Scatter(
                x=df[df['origin']=='production']['odometer'],
                y=df[df['origin']=='production']['soh'],
                mode="markers",
                marker=dict(color="orange", size=8, opacity=0.7),
                name="Production",
            )
        )
    
    
    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title="Odometer (km)",
        yaxis_title="State of Health (SoH)",
        legend_title="Source",
        title=f"version: {model}",
        template="plotly_white",
        xaxis=dict(
            range=[0, 150000]
        ),  # Change selon l'échelle souhaitée pour l'odomètre
        yaxis=dict(range=[0.75, 1.1]),  # Change selon l'échelle souhaitée pour le SoH
    )
    return fig

In [ ]:
for model in trendline_prod.model_name.unique():
    if model  not in  ['model 3', 'model y', 'model s', 'model x']:
        for type_car in trendline_prod[trendline_prod['model_name']==model]['type'].unique():
            df_temp = trendline_prod[(trendline_prod['model_name']==model) & (trendline_prod['type']==type_car)]
            data = df[(df['model_name']==model) & (df['type']==type_car)].reset_index(drop=True)
            print(filter_trendlines(df_temp['trendline'].values[0], df_temp['trendline_max'].values[0], df_temp['trendline_min'].values[0]))
            fig = show_trendline_model(df_temp['trendline'].values[0], df_temp['trendline_max'].values[0], df_temp['trendline_min'].values[0], f'{model} {type_car}', df=data)
            fig.show()